In [1]:
from finn.util.basic import make_build_dir
from finn.util.visualization import showInNetron
from qonnx.core.modelwrapper import ModelWrapper
import os
from os.path import join
import shutil
    
build_dir = os.environ["FINN_BUILD_DIR"]
finn_root = os.environ["FINN_ROOT"]
build_dir
# finn_root

'/scratch/users/mdaniowi/build_dirs/yolov8'

In [2]:
!pip install opencv-python
!pip install py-cpuinfo

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 2.7 MB/s eta 0:00:0000:0100:01
Defaulting to user installation because normal site-packages is not writeable


In [4]:
from ultralytics import YOLO

net = YOLO('quantyolov8.yaml').load('yolov8n.pt').model
# net = YOLO('quantyolov8.yaml').model
print(type(net))

WARNING ⚠️ no model scale passed. Assuming scale='n'.
Transferred 355/355 items from pretrained weights
<class 'ultralytics.nn.tasks.DetectionModel'>


In [6]:
# QONNX EXPORT

import torch
import numpy as np
from brevitas.export import export_qonnx

from qonnx.core.datatype import DataType
from qonnx.util.cleanup import cleanup_model
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.fold_constants import FoldConstants
from qonnx.transformation.general import (
    GiveReadableTensorNames,
    GiveUniqueNodeNames,
    RemoveStaticGraphInputs,
)
from onnx import helper as oh

from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN
from ultralytics.nn.modules import C2f, QuantC2f, QuantDetect


# test input
test_input_np = (np.random.rand(1, 3, 640, 480) * 255).astype(np.uint8)
# np.save(join(finn_root, "notebooks", "experiments", "yolov8", "test_input_640x480.npy"), test_input_np)
test_input_np = np.load(join(finn_root, "notebooks", "experiments", "yolov8", "test_input_640x480.npy"))
test_input = torch.from_numpy(test_input_np).float()

net = net.eval()
for m in net.modules():
    if isinstance(m, C2f) or isinstance(m, QuantC2f):
        m.forward = m.forward_split
    if isinstance(m, QuantDetect):
        m.finn_export = True

output_golden = net(test_input)
for i, out in enumerate(output_golden):
    np.save(join(finn_root, "notebooks", "experiments", "yolov8", "output_golden_{}.npy".format(i)), out.detach())

# onnx export
model_file = join(finn_root, "notebooks", "experiments", "yolov8", "quantyolov8.onnx")
onnx_model = export_qonnx(net, test_input, model_file)
model = ModelWrapper(model_file)
model = cleanup_model(model)
model = model.transform(ConvertQONNXtoFINN())

# assume input to be uint8
global_inp_name = model.graph.input[0].name
model.set_tensor_datatype(global_inp_name, DataType["UINT8"])
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(InferDataTypes())
model = model.transform(RemoveStaticGraphInputs())
model.save(model_file)
# model.save(join(finn_root, "notebooks", "experiments", "yolov8", "quantyolov8_qonnx.onnx"))

shape not specified: /model.0/conv/Conv_output_0
shape not specified: /model.0/conv/Conv_output_0
shape not specified: model.0.bn.weight
shape not specified: model.0.bn.bias
shape not specified: model.0.bn.running_mean
shape not specified: model.0.bn.running_var
shape not specified: /model.0/bn/BatchNormalization_output_0
shape not specified: /model.0/bn/BatchNormalization_output_0
shape not specified: /model.0/act/act_quant/activation_impl/Relu_output_0
shape not specified: /model.0/act/act_quant/activation_impl/Relu_output_0
shape not specified: /model.0/act/act_quant/export_handler/Constant_output_0
shape not specified: /model.0/conv/weight_quant/export_handler/Constant_3_output_0
shape not specified: /model.0/conv/weight_quant/export_handler/Constant_output_0
shape not specified: /model.0/act/act_quant/export_handler/Quant_output_0
shape not specified: /model.1/conv/weight_quant/export_handler/Constant_1_output_0
shape not specified: /model.1/conv/weight_quant/export_handler/Consta

In [22]:
# QONNX VERIFICATION
import numpy as np
import finn.core.onnx_exec as oxe
from onnx import helper as oh
import onnx

# print(len(output_golden))
# print([x.shape for x in output_golden])
test_input_np = np.load(join(finn_root, "notebooks", "experiments", "yolov8", "test_input_640x480.npy"))
output_golden = [np.load(join(finn_root, "notebooks", "experiments", "yolov8", "output_golden_{}.npy".format(i))) for i in range(3)]

model_file = join(finn_root, "notebooks", "experiments", "yolov8", "quantyolov8.onnx")
input_dict = {"global_in": test_input_np.astype(np.float32)}
qonnx_model = ModelWrapper(model_file)
dummy = oh.make_tensor_value_info("dummy", onnx.TensorProto.FLOAT, [1])
for n in qonnx_model.graph.node:
    if n.op_type == "Resize":
        print(n.input)
        # n.input[:] = [n.input[0], n.input[2]]
        n.input[1] = dummy.name

qonnx_model.graph.value_info.append(dummy)
qonnx_model.save('tescik.onnx')
output_dict = oxe.execute_onnx(qonnx_model, input_dict, return_full_exec_context=False)

input shape not specified:  Resize_0
input shape not specified:  Resize_1
['Mul_53_out0', '', 'Resize_0_param0']
['Mul_61_out0', '', 'Resize_1_param0']


In [28]:
for i, (k, v) in enumerate(output_dict.items()):
    print(v.shape, output_golden[i].shape, np.max(np.abs(v - output_golden[i])))
    print("output:")
    print(v)
    print('golden:')
    print(output_golden[i])



(1, 144, 80, 60) (1, 144, 80, 60) 0.00091791153
output:
[[[[  2.9704547   2.783561    2.2772334 ...   2.2493327   2.336242
      2.4333029]
   [  3.2602565   2.923632    2.886554  ...   2.884255    2.9436705
      2.7787452]
   [  3.2754226   2.9353542   3.0400503 ...   3.021989    3.0048552
      2.8399205]
   ...
   [  3.2411072   2.9416175   3.0611484 ...   2.9832177   2.9495835
      2.831907 ]
   [  3.1815596   2.8327017   2.92665   ...   2.8611512   2.767042
      2.737022 ]
   [  3.153782    2.9408984   2.8025966 ...   2.7752163   2.6836994
      2.7344203]]

  [[  3.411058    3.3295212   2.8624642 ...   2.8461568   2.9280748
      3.068747 ]
   [  3.6102362   3.4458182   3.3198514 ...   3.2939217   3.421071
      3.3648825]
   [  3.6877484   3.6381683   3.4060225 ...   3.4141288   3.569401
      3.4392762]
   ...
   [  3.6377964   3.558186    3.37275   ...   3.3735988   3.515358
      3.37995  ]
   [  3.5572705   3.3879797   3.2969255 ...   3.330885    3.4287481
      3.3326397

In [3]:
# model_file = join(finn_root, "notebooks", "experiments", "yolov8", "quantyolov8.onnx")
showInNetron(model_file, "xirengcto03")

Serving '/scratch/users/mdaniowi/finn/notebooks/experiments/yolov8/quantyolov8.onnx' at http://0.0.0.0:2222


In [21]:
# model_file = join(finn_root, "notebooks", "experiments", "yolov8", "quantyolov8.onnx")
showInNetron("tescik.onnx", "xirengcto03")

Stopping http://0.0.0.0:2222
Serving 'tescik.onnx' at http://0.0.0.0:2222


In [7]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg

from qonnx.transformation.base import Transformation
from finn.transformation.streamline.reorder import (
    MoveLinearPastEltwiseAdd,
    MoveLinearPastFork,
)


# def step_streamline_nonlinear(model: ModelWrapper, cfg: build.DataflowBuildConfig):
#     streamline_transformations = [
#         MoveLinearPastEltwiseAdd(),
#         MoveLinearPastFork(),
#     ]
#     for trn in streamline_transformations:
#         print('hello')
#         model = model.transform(trn)
#         model = model.transform(GiveUniqueNodeNames())
#     return model



model_file = join(finn_root, "notebooks", "experiments", "yolov8", "quantyolov8.onnx")
output_dir = join(build_dir, "output_dir")
intermediate_models = join(output_dir, "intermediate_models")
# folding_config_file = join(finn_root, "notebooks", "experiments", "tfc_2mvau", "folding_intrtl_extrtl.json")
# specialize_layers_config_file = join(finn_root, "notebooks", "experiments", "tfc_2mvau", "specialize_rtl_rtl.json")

build_dataflow_steps = [
    "step_qonnx_to_finn",
    "step_tidy_up",
    "step_streamline",
    # step_streamline_nonlinear,
    # MoveLinearOpsPastSplit,
    # "step_convert_to_hw",
    # "step_create_dataflow_partition",
    # "step_specialize_layers",
    # "step_target_fps_parallelization",
    # "step_apply_folding_config",
    # "step_minimize_bit_width",
    # "step_generate_estimate_reports",
    # "step_hw_codegen",
    # "step_hw_ipgen",
    # "step_set_fifo_depths",
    # "step_create_stitched_ip",
    # "step_measure_rtlsim_performance",
    # "step_out_of_context_synthesis",
    # "step_synthesize_bitfile",
    # "step_make_pynq_driver",
    # "step_deployment_package",
]

#Delete previous run results if exist
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
    print("Previous run results deleted!")
    
cfg = build.DataflowBuildConfig(
    output_dir=output_dir,
    verbose=True,
    standalone_thresholds=True,
    # folding_config_file=folding_config_file,
    # specialize_layers_config_file=specialize_layers_config_file,
    synth_clk_period_ns=10,
    board="ZCU104",
    shell_flow_type=build_cfg.ShellFlowType.VIVADO_ZYNQ,
    generate_outputs=[
        build_cfg.DataflowOutputType.ESTIMATE_REPORTS,
        build_cfg.DataflowOutputType.BITFILE,
        build_cfg.DataflowOutputType.PYNQ_DRIVER,
        build_cfg.DataflowOutputType.DEPLOYMENT_PACKAGE,
    ],
    steps = build_dataflow_steps
)
build.build_dataflow_cfg(model_file, cfg)

Previous run results deleted!
Building dataflow accelerator from /scratch/users/mdaniowi/finn/notebooks/experiments/yolov8/quantyolov8.onnx
Intermediate outputs will be generated in /scratch/users/mdaniowi/build_dirs/yolov8
Final outputs will be generated in /scratch/users/mdaniowi/build_dirs/yolov8/output_dir
Build log is at /scratch/users/mdaniowi/build_dirs/yolov8/output_dir/build_dataflow.log
Running step: step_qonnx_to_finn [1/3]
Running step: step_tidy_up [2/3]
Running step: step_streamline [3/3]
Completed successfully


/scratch/users/mdaniowi/finn/deps/qonnx/src/qonnx/transformation/infer_data_layouts.py:127: UserWarning: Assuming 4D input is NCHW
  warnings.warn("Assuming 4D input is NCHW")


0

In [2]:
# STREAMLINE

from copy import deepcopy

import numpy as np

from qonnx.transformation.base import Transformation
from qonnx.transformation.general import(
    SortGraph,
    RemoveUnusedTensors,
)
from qonnx.util.basic import get_by_name
from qonnx.transformation.infer_data_layouts import InferDataLayouts
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.lower_convs_to_matmul import LowerConvsToMatMul
from qonnx.transformation.general import (
    GiveReadableTensorNames,
    GiveUniqueNodeNames,
    RemoveStaticGraphInputs,
)

from finn.transformation.streamline import Streamline
import finn.transformation.streamline.absorb as absorb
from finn.transformation.streamline.reorder import (
    MoveLinearPastEltwiseAdd,
    MoveLinearPastFork,
    MoveTransposePastFork,
    MoveTransposePastJoinAdd,
    MoveMulPastJoinAdd,
    MoveOpPastFork,
    MoveScalarMulPastConv,
    # MoveIdenticalOpPastJoinOp,

    MoveScalarLinearPastSplit,
    MoveTransposePastSplit,
    MoveMulPastJoinConcat,
    MoveAddPastJoinConcat,
    MoveTransposePastJoinConcat,

    MoveMulPastMaxPool,
    MakeMaxPoolNHWC,
    MakeScaleResizeNHWC
)

model_file = join(finn_root, "notebooks", "experiments", "yolov8", "quantyolov8.onnx")
output_dir = join(build_dir, "output_dir")
intermediate_models = join(output_dir, "intermediate_models")

# model = ModelWrapper(join(intermediate_models, "step_tidy_up.onnx"))
model = ModelWrapper(model_file)
model = model.transform(Streamline())
model = model.transform(MoveAddPastJoinConcat())
model = model.transform(InferDataLayouts())
model = model.transform(RemoveUnusedTensors())
model.save(join(build_dir, "test.onnx"))

# ---------------- shuffle affine ops 
# forks:
model = model.transform(MoveScalarLinearPastSplit())
#
model = model.transform(MoveLinearPastFork())
model = model.transform(InferDataTypes())
model = model.transform(InferDataLayouts())
model = model.transform(GiveUniqueNodeNames())
model.save(join(build_dir, "test1.onnx"))
# joins:

# model = model.transform(MoveLinearPastEltwiseAdd(), cleanup=True)
model = model.transform(MoveMulPastJoinAdd())
model = model.transform(InferDataTypes())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model.save(join(build_dir, "test2.onnx"))
#
model = model.transform(MoveMulPastJoinConcat())
model.save(join(build_dir, "test3.onnx"))
#
model = model.transform(Streamline())
# deal with SPPF
model = model.transform(MoveLinearPastFork())
model = model.transform(MoveMulPastMaxPool())
model = model.transform(MoveLinearPastFork())
model = model.transform(MoveMulPastMaxPool())
model = model.transform(MoveMulPastJoinConcat())
model = model.transform(Streamline())
model.save(join(build_dir, "test4.onnx"))


# ------------------ shuffle Transpose
# forks:
model = model.transform(LowerConvsToMatMul())
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
model = model.transform(absorb.AbsorbConsecutiveTransposes())
model.save(join(build_dir, "test5.onnx"))
model = model.transform(MakeScaleResizeNHWC())
model.save(join(build_dir, "test5_5.onnx"))
#
model = model.transform(MoveTransposePastSplit())
#
model = model.transform(MoveTransposePastFork())
model = model.transform(InferDataLayouts())
model = model.transform(InferDataTypes())
model.save(join(build_dir, "test6.onnx"))
# joins:
model = model.transform(MoveTransposePastJoinAdd())
model = model.transform(InferDataLayouts())
model = model.transform(InferDataTypes())
model.save(join(build_dir, "test7.onnx"))
#
model = model.transform(MoveTransposePastJoinConcat())
model.save(join(build_dir, "test8.onnx"))
#
model = model.transform(absorb.AbsorbConsecutiveTransposes())
model = model.transform(InferDataTypes())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model.save(join(build_dir, "test9.onnx"))
#
# SPPF
model = model.transform(MakeMaxPoolNHWC())
model = model.transform(MoveTransposePastJoinConcat())
model = model.transform(absorb.AbsorbConsecutiveTransposes())


model.save(join(build_dir, "streamlined.onnx"))

/scratch/users/mdaniowi/finn/deps/qonnx/src/qonnx/transformation/infer_data_layouts.py:127: UserWarning: Assuming 4D input is NCHW
  warnings.warn("Assuming 4D input is NCHW")
/scratch/users/mdaniowi/finn/src/finn/transformation/streamline/reorder.py:1369: UserWarning: Producers not identical, skipping
  warnings.warn("Producers not identical, skipping")
/scratch/users/mdaniowi/finn/src/finn/transformation/streamline/absorb.py:66: UserWarning: Threshold or add bias not constant, skipping
  warnings.warn("Threshold or add bias not constant, skipping")
/scratch/users/mdaniowi/finn/deps/qonnx/src/qonnx/core/modelwrapper.py:382: UserWarning: find_consumer: found multiple consumers, returning first one
  warnings.warn("find_consumer: found multiple consumers, returning first one")


In [6]:
showInNetron(join(finn_root, "notebooks", "experiments", "yolov8", "quantyolov8.onnx"), "xirengcto03")

Stopping http://0.0.0.0:2222
Serving '/scratch/users/mdaniowi/finn/notebooks/experiments/yolov8/quantyolov8.onnx' at http://0.0.0.0:2222


In [11]:
showInNetron(join(intermediate_models, "test2.onnx"), "xirengcto03")

Stopping http://0.0.0.0:2222
Serving '/scratch/users/mdaniowi/build_dirs/yolov8/output_dir/intermediate_models/step_streamline.onnx' at http://0.0.0.0:2222


In [3]:
showInNetron(join(build_dir, "test.onnx"), "xirengcto03")

Serving '/scratch/users/mdaniowi/build_dirs/yolov8/test.onnx' at http://0.0.0.0:2222


In [7]:
showInNetron(join(build_dir, "test4.onnx"), "xirengcto03")

Stopping http://0.0.0.0:2222
Serving '/scratch/users/mdaniowi/build_dirs/yolov8/test4.onnx' at http://0.0.0.0:2222


In [5]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
import shutil




import json
import numpy as np
import os
import shutil
import warnings
from copy import deepcopy
from distutils.dir_util import copy_tree
from functools import partial
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.custom_op.registry import getCustomOp
from qonnx.transformation.bipolar_to_xnor import ConvertBipolarMatMulToXnorPopcount
from qonnx.transformation.fold_constants import FoldConstants
from qonnx.transformation.general import (
    ApplyConfig,
    GiveReadableTensorNames,
    GiveUniqueNodeNames,
    RemoveStaticGraphInputs,
    RemoveUnusedTensors,
)
from qonnx.transformation.infer_data_layouts import InferDataLayouts
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.lower_convs_to_matmul import LowerConvsToMatMul
from qonnx.util.cleanup import cleanup_model
from qonnx.util.config import extract_model_config_to_json
from shutil import copy

import finn.transformation.fpgadataflow.convert_to_hw_layers as to_hw
import finn.transformation.streamline.absorb as absorb
from finn.analysis.fpgadataflow.dataflow_performance import dataflow_performance
from finn.analysis.fpgadataflow.exp_cycles_per_layer import exp_cycles_per_layer
from finn.analysis.fpgadataflow.hls_synth_res_estimation import hls_synth_res_estimation
from finn.analysis.fpgadataflow.op_and_param_counts import (
    aggregate_dict_keys,
    op_and_param_counts,
)
from finn.analysis.fpgadataflow.post_synth_res import post_synth_res
from finn.analysis.fpgadataflow.res_estimation import (
    res_estimation,
    res_estimation_complete,
)
from finn.builder.build_dataflow_config import (
    DataflowBuildConfig,
    DataflowOutputType,
    ShellFlowType,
    VerificationStepType,
)
from finn.core.onnx_exec import execute_onnx
from finn.core.rtlsim_exec import rtlsim_exec
from finn.core.throughput_test import throughput_test_rtlsim
from finn.transformation.fpgadataflow.annotate_cycles import AnnotateCycles
from finn.transformation.fpgadataflow.compile_cppsim import CompileCppSim
from finn.transformation.fpgadataflow.create_dataflow_partition import (
    CreateDataflowPartition,
)
from finn.transformation.fpgadataflow.create_stitched_ip import CreateStitchedIP
from finn.transformation.fpgadataflow.derive_characteristic import (
    DeriveCharacteristic,
    DeriveFIFOSizes,
)
from finn.transformation.fpgadataflow.hlssynth_ip import HLSSynthIP
from finn.transformation.fpgadataflow.insert_dwc import InsertDWC
from finn.transformation.fpgadataflow.insert_fifo import InsertFIFO
from finn.transformation.fpgadataflow.make_pynq_driver import MakePYNQDriver
from finn.transformation.fpgadataflow.make_zynq_proj import ZynqBuild
from finn.transformation.fpgadataflow.minimize_accumulator_width import (
    MinimizeAccumulatorWidth,
)
from finn.transformation.fpgadataflow.minimize_weight_bit_width import (
    MinimizeWeightBitWidth,
)
from finn.transformation.fpgadataflow.prepare_cppsim import PrepareCppSim
from finn.transformation.fpgadataflow.prepare_ip import PrepareIP
from finn.transformation.fpgadataflow.prepare_rtlsim import PrepareRTLSim
from finn.transformation.fpgadataflow.replace_verilog_relpaths import (
    ReplaceVerilogRelPaths,
)
from finn.transformation.fpgadataflow.set_exec_mode import SetExecMode
from finn.transformation.fpgadataflow.set_fifo_depths import (
    InsertAndSetFIFODepths,
    RemoveShallowFIFOs,
    SplitLargeFIFOs,
)
from finn.transformation.fpgadataflow.set_folding import SetFolding
from finn.transformation.fpgadataflow.specialize_layers import SpecializeLayers
from finn.transformation.fpgadataflow.synth_ooc import SynthOutOfContext
from finn.transformation.fpgadataflow.vitis_build import VitisBuild
from finn.transformation.move_reshape import RemoveCNVtoFCFlatten
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN
from finn.transformation.qonnx.quant_act_to_multithreshold import (
    default_filter_function_generator,
)
from finn.transformation.streamline import Streamline
from finn.transformation.streamline.reorder import MakeMaxPoolNHWC


def my_streamline(model: ModelWrapper, cfg: build.DataflowBuildConfig):
    """Run streamlining on given model. Streamlining involves moving floating point
    scale/shift parameters around, collapsing adjacent ones into a single parameter,
    then absorbing the scale/shift into the following `MultiThreshold` node.
    Streamlining requires careful topology design and cannot be applied to all
    topologies.
    """

    model = model.transform(absorb.AbsorbSignBiasIntoMultiThreshold())
    model = model.transform(Streamline())
    need_lowering = len(model.get_nodes_by_op_type("Conv")) > 0
    if need_lowering:
        model = model.transform(LowerConvsToMatMul())
        model = model.transform(MakeMaxPoolNHWC())
        model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
        model = model.transform(MakeMaxPoolNHWC())
        model = model.transform(absorb.AbsorbConsecutiveTransposes())
    # model = model.transform(ConvertBipolarMatMulToXnorPopcount())
    # model = model.transform(Streamline())
    # # absorb final add-mul nodes into TopK
    # model = model.transform(absorb.AbsorbScalarMulAddIntoTopK())
    # model = model.transform(InferDataLayouts())
    # model = model.transform(RemoveUnusedTensors())

    if VerificationStepType.STREAMLINED_PYTHON in cfg._resolve_verification_steps():
        verify_step(model, cfg, "streamlined_python", need_parent=False)

    return model


estimates_output_dir = build_dir + "/output_estimates_only"
estimate_only_dataflow_steps = [
    "step_qonnx_to_finn",
    "step_tidy_up",
    # "step_streamline",
    my_streamline,
    "step_convert_to_hw",
    # "step_create_dataflow_partition",
    "step_specialize_layers",
    "step_target_fps_parallelization",
    "step_apply_folding_config",
    "step_minimize_bit_width",
    "step_generate_estimate_reports",
]

#Delete previous run results if exist
if os.path.exists(estimates_output_dir):
    shutil.rmtree(estimates_output_dir)
    print("Previous run results deleted!")


cfg_estimates = build.DataflowBuildConfig(
    output_dir          = estimates_output_dir,
    mvau_wwidth_max     = 10000,
    target_fps          = 30,
    synth_clk_period_ns = 10.0,
    # fpga_part           = "xc7z020clg400-1",
    board               = "U250",
    steps               = estimate_only_dataflow_steps,
    shell_flow_type=build_cfg.ShellFlowType.VITIS_ALVEO,
    generate_outputs=[
        build_cfg.DataflowOutputType.ESTIMATE_REPORTS,
    ]
)
build.build_dataflow_cfg(model_file, cfg_estimates)

Previous run results deleted!
Building dataflow accelerator from /scratch/users/mdaniowi/finn/notebooks/experiments/yolov8/quantyolov8.onnx
Intermediate outputs will be generated in /scratch/users/mdaniowi/build_dirs/yolov8
Final outputs will be generated in /scratch/users/mdaniowi/build_dirs/yolov8/output_estimates_only
Build log is at /scratch/users/mdaniowi/build_dirs/yolov8/output_estimates_only/build_dataflow.log
Running step: step_qonnx_to_finn [1/9]
Running step: step_tidy_up [2/9]
Running step: my_streamline [3/9]
Running step: step_convert_to_hw [4/9]
Running step: step_specialize_layers [5/9]
Running step: step_target_fps_parallelization [6/9]


Traceback (most recent call last):
  File "/scratch/users/mdaniowi/finn/src/finn/builder/build_dataflow.py", line 158, in build_dataflow_cfg
    model = transform_step(model, cfg)
  File "/scratch/users/mdaniowi/finn/src/finn/builder/build_dataflow_steps.py", line 421, in step_target_fps_parallelization
    model = model.transform(
  File "/scratch/users/mdaniowi/finn/deps/qonnx/src/qonnx/core/modelwrapper.py", line 140, in transform
    (transformed_model, model_was_changed) = transformation.apply(transformed_model)
  File "/scratch/users/mdaniowi/finn/src/finn/transformation/fpgadataflow/set_folding.py", line 226, in apply
    perf_dict = model.analysis(dataflow_performance)
  File "/scratch/users/mdaniowi/finn/deps/qonnx/src/qonnx/core/modelwrapper.py", line 124, in analysis
    return analysis_fxn(self)
  File "/scratch/users/mdaniowi/finn/src/finn/analysis/fpgadataflow/dataflow_performance.py", line 71, in dataflow_performance
    max_pred_latency = max(pred_latencies)
  File "/sc

> /scratch/users/mdaniowi/finn/src/finn/analysis/fpgadataflow/dataflow_performance.py(70)<lambda>()
     68                 else:
     69                     # find max of any of predecessors
---> 70                     pred_latencies = map(lambda x: latency_at_node_output[x.name], predecessors)
     71                     max_pred_latency = max(pred_latencies)
     72                 latency_at_node_output[node.name] = node_cycles + max_pred_latency



ipdb>  exit


Build failed


-1

In [3]:
showInNetron(join(build_dir, "test5.onnx"), "xirengcto03")

Serving '/scratch/users/mdaniowi/build_dirs/yolov8/test5.onnx' at http://0.0.0.0:2222


In [4]:
showInNetron(join(build_dir, "test5_5.onnx"), "xirengcto03")

Stopping http://0.0.0.0:2222
Serving '/scratch/users/mdaniowi/build_dirs/yolov8/test5_5.onnx' at http://0.0.0.0:2222


In [4]:
showInNetron(join(build_dir, "test9.onnx"), "xirengcto03")

Stopping http://0.0.0.0:2222
Serving '/scratch/users/mdaniowi/build_dirs/yolov8/test9.onnx' at http://0.0.0.0:2222


In [4]:
showInNetron(join(build_dir, "streamlined.onnx"), "xirengcto03")

Stopping http://0.0.0.0:2222
Serving '/scratch/users/mdaniowi/build_dirs/yolov8/streamlined.onnx' at http://0.0.0.0:2222


In [21]:
# CONVERT TO HW
import finn.transformation.fpgadataflow.convert_to_hw_layers as to_hw


model = ModelWrapper(join(build_dir, "streamlined.onnx"))

standalone_thresholds = True
if standalone_thresholds:
    # doing this first causes all threshold layers to be standalone
    model = model.transform(to_hw.InferThresholdingLayer())
# needed for bipolar MatMul layers
# model = model.transform(to_hw.InferBinaryMatrixVectorActivation())
# needed for non-bipolar MatMul layers
model = model.transform(to_hw.InferQuantizedMatrixVectorActivation())
# TopK to LabelSelect
# model = model.transform(to_hw.InferLabelSelectLayer())
# input quantization (if any) as standalone threshold
model = model.transform(to_hw.InferThresholdingLayer())
# needed for convolutions -- TODO always exec?
need_conv = len(model.get_nodes_by_op_type("Im2Col")) > 0
if need_conv:
    model = model.transform(to_hw.InferConvInpGen())
    model = model.transform(to_hw.InferStreamingMaxPool())
    # model = model.transform(RemoveCNVtoFCFlatten())
# get rid of Tranpose -> Tranpose identity seq
model = model.transform(absorb.AbsorbConsecutiveTransposes())

#new
model = model.transform(to_hw.InferAddStreamsLayer())
model = model.transform(to_hw.InferConcatLayer())

model = model.transform(GiveUniqueNodeNames())
model = model.transform(InferDataLayouts())

model.save(join(build_dir, "converted_to_hw.onnx"))

In [22]:
showInNetron(join(build_dir, "converted_to_hw.onnx"), "xirengcto03")

Stopping http://0.0.0.0:4444
Serving '/scratch/users/mdaniowi/build_dirs/yolov8/converted_to_hw.onnx' at http://0.0.0.0:4444
